In [1]:
import copy
import math
import random
import time
from collections import OrderedDict, defaultdict
from typing import Union, List
from utils import *

import numpy as np
import torch
import torch.nn as nn
from torch.optim import *
from torch.utils.data import DataLoader
from torchprofile import profile_macs
from tqdm.auto import tqdm
import pickle

import matplotlib.pyplot as plt
import timm
from collapsible_MLP import *

/home/soheil/miniconda3/envs/lc/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
num_classes = 100

In [3]:
model = timm.models.mlp_mixer.MlpMixer(num_classes=num_classes, patch_size=4, img_size=32, drop_rate=0.1, mlp_layer=CollapsibleMlp)

In [4]:
model

MlpMixer(
  (stem): PatchEmbed(
    (proj): Conv2d(3, 512, kernel_size=(4, 4), stride=(4, 4))
    (norm): Identity()
  )
  (blocks): Sequential(
    (0): MixerBlock(
      (norm1): LayerNorm((512,), eps=1e-06, elementwise_affine=True)
      (mlp_tokens): CollapsibleMlp(
        (fc1): Linear(in_features=64, out_features=256, bias=True)
        (act): PReLU(num_parameters=1)
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity()
        (fc2): Linear(in_features=256, out_features=64, bias=True)
        (drop2): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((512,), eps=1e-06, elementwise_affine=True)
      (mlp_channels): CollapsibleMlp(
        (fc1): Linear(in_features=512, out_features=2048, bias=True)
        (act): PReLU(num_parameters=1)
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity()
        (fc2): Linear(in_features=2048, out_features=512, bias=True)
        (drop2): Dropout(p=0.0, inplace

In [31]:
def get_model_linear_loss(model, fraction=1.0):
    linear_loss = 0
    num_mlp_layers = len(list(model.named_modules()))
    for name, module in list(model.named_modules())[::-1][:int(num_mlp_layers * fraction)]:
        if isinstance(module, CollapsibleMlp):
            linear_loss += module.linear_loss()
            print(linear_loss)
    return linear_loss

In [34]:
get_model_linear_loss(model, fraction=.2)

tensor([0.8100], grad_fn=<AddBackward0>)
tensor([1.6200], grad_fn=<AddBackward0>)
tensor([2.4300], grad_fn=<AddBackward0>)


tensor([2.4300], grad_fn=<AddBackward0>)

In [20]:
for name, module in reversed(list(model.named_modules())):
    print("---",name)
    print(module)

--- head
Linear(in_features=512, out_features=100, bias=True)
--- head_drop
Dropout(p=0.1, inplace=False)
--- norm
LayerNorm((512,), eps=1e-06, elementwise_affine=True)
--- blocks.7.mlp_channels.drop2
Dropout(p=0.0, inplace=False)
--- blocks.7.mlp_channels.fc2
Linear(in_features=2048, out_features=512, bias=True)
--- blocks.7.mlp_channels.norm
Identity()
--- blocks.7.mlp_channels.drop1
Dropout(p=0.0, inplace=False)
--- blocks.7.mlp_channels.act
PReLU(num_parameters=1)
--- blocks.7.mlp_channels.fc1
Linear(in_features=512, out_features=2048, bias=True)
--- blocks.7.mlp_channels
CollapsibleMlp(
  (fc1): Linear(in_features=512, out_features=2048, bias=True)
  (act): PReLU(num_parameters=1)
  (drop1): Dropout(p=0.0, inplace=False)
  (norm): Identity()
  (fc2): Linear(in_features=2048, out_features=512, bias=True)
  (drop2): Dropout(p=0.0, inplace=False)
)
--- blocks.7.norm2
LayerNorm((512,), eps=1e-06, elementwise_affine=True)
--- blocks.7.drop_path
Identity()
--- blocks.7.mlp_tokens.drop2
